# 聚类的评估指标

聚类的评估不像分类那么容易,我们训练的时候只知道特征没有标签,那自然评估的时候和标签会对不上.下面是几种常见的聚类性能评估方式

## Adjusted Rand  Index 指数



$$ \text{RI} = \frac{a + b}{C_2^{n_{samples}}} $$


其中 $C_2^{n_{samples}}$ 是数据集中可能的数据对(pairs)的总数(不排序).

然而,RI评分不能保证随机标签任务(random label assignments)将获得接近零的值(特别是如果簇的数量与采样数量相同的数量级).


为了抵消这种影响,我们可以通过定义`adjusted Rand index`(调整后的Rand index)来折现(discount)随机标签的预期$RI E[\text{RI}]$ ,如下所示:

$$ \text{ARI} = \frac{\text{RI} - E[\text{RI}]}{\max(\text{RI}) - E[\text{RI}]}$$

`adjusted rand score`是对称的(symmetric)-- 交换参数不会改变得分.它可以作为共识度量(consensus measure),

完美的标签得分为 1.0,而负数或者得分接近0说明效果不佳.